In [2]:
%cd ../model/

/home/chisphung/ALPR_Fisheye/model


In [3]:
!pwd

/home/chisphung/ALPR_Fisheye/model


In [4]:
import numpy as np
import torch
import os
import sys
import argparse
from yolo.ultralytics.utils.nms import non_max_suppression
# from ai_core.object_detection.yolov5_custom.od.data.datasets import letterbox
from typing import List
# from dynaconf import settings
from yolo.ultralytics.models.yolo.model import YOLO
import cv2

In [5]:
class Detection:
    def __init__(self, weights_path='.pt',size=(640,640),device='cpu',iou_thres=None,conf_thres=None):
        # cwd = os.path.dirname(__file__)
        self.device=device
        self.char_model, self.names = self.load_model(weights_path)
        self.size=size
        
        self.iou_thres=iou_thres
        self.conf_thres=conf_thres

    def detect(self, frame):
        
        results, resized_img = self.char_detection_yolo(frame)

        return results, resized_img
    
    def preprocess_image(self, original_image):

        resized_img = self.ResizeImg(original_image,size=self.size)
        image = resized_img.copy()[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        image = np.ascontiguousarray(image)

        image = torch.from_numpy(image).to(self.device)
        image = image.float()
        image = image / 255.0
        if image.ndimension() == 3:
            image = image.unsqueeze(0)
        return image, resized_img
    
    def char_detection_yolo(self, image, classes=None, \
                            agnostic_nms=True, max_det=1000):

        resized_img = self.ResizeImg(image.copy(), size=self.size)
        
        # Use YOLO model directly with YOLOv11 format
        results = self.char_model.predict(
            resized_img,
            conf=self.conf_thres,
            iou=self.iou_thres,
            classes=classes,
            agnostic_nms=agnostic_nms,
            max_det=max_det,
            verbose=False
        )
        
        detection_results = []
        
        # Process YOLOv11 results format
        for result in results:
            if result.boxes is not None:
                boxes = result.boxes
                for i in range(len(boxes)):
                    # Get box coordinates (xyxy format)
                    xyxy = boxes.xyxy[i].cpu().numpy()
                    conf = boxes.conf[i].cpu().numpy()
                    cls = int(boxes.cls[i].cpu().numpy())
                    
                    # Format: [class_name, confidence, (x1, y1, x2, y2)]
                    detection_result = [self.names[cls], str(float(conf)), (float(xyxy[0]), float(xyxy[1]), float(xyxy[2]), float(xyxy[3]))]
                    detection_results.append(detection_result)
        
        return detection_results, resized_img
        
    def ResizeImg(self, img, size):
        h1, w1, _ = img.shape
        # print(h1, w1, _)
        h, w = size
        if w1 < h1 * (w / h):
            # print(w1/h1)
            img_rs = cv2.resize(img, (int(float(w1 / h1) * h), h))
            mask = np.zeros((h, w - (int(float(w1 / h1) * h)), 3), np.uint8)
            img = cv2.hconcat([img_rs, mask])
            trans_x = int(w / 2) - int(int(float(w1 / h1) * h) / 2)
            trans_y = 0
            trans_m = np.float32([[1, 0, trans_x], [0, 1, trans_y]])
            height, width = img.shape[:2]
            img = cv2.warpAffine(img, trans_m, (width, height))
            return img
        else:
            img_rs = cv2.resize(img, (w, int(float(h1 / w1) * w)))
            mask = np.zeros((h - int(float(h1 / w1) * w), w, 3), np.uint8)
            img = cv2.vconcat([img_rs, mask])
            trans_x = 0
            trans_y = int(h / 2) - int(int(float(h1 / w1) * w) / 2)
            trans_m = np.float32([[1, 0, trans_x], [0, 1, trans_y]])
            height, width = img.shape[:2]
            img = cv2.warpAffine(img, trans_m, (width, height))
            return img
    def load_model(self,path, train = False):
        # print(self.device)
        model = YOLO(path)  # load FP32 model
        names = model.module.names if hasattr(model, 'module') else model.names  # get class names
        if train:
            model.train()
        else:
            model.eval()
        return model, names
    def xyxytoxywh(self, x):
        # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] where xy1=top-left, xy2=bottom-right
        y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
        y[0] = (x[0] + x[2]) / 2  # x center
        y[1] = (x[1] + x[3]) / 2  # y center
        y[2] = x[2] - x[0]  # width
        y[3] = x[3] - x[1]  # height
        return y

In [6]:
char_model=Detection(size=(640,640),weights_path='../weight/recognition_v11m.pt',device='cuda:0',iou_thres=0.5,conf_thres=0.1)


In [7]:
img_names=os.listdir('../dataset/test')
img_names

['Japanese-20License-20Plates-20-20Custom-20Japanese-20License-20P_jpg.rf.66abdc04fa057e3e07e95fab5caee67e.jpg',
 'xemay712_jpg.rf.c27e1277daa8f12d61c4d1e75377a3f0.jpg',
 'xemay2068_jpg.rf.d1cf718d5a7d6e7c335c386c89b362c2.jpg',
 'xemay2080_jpg.rf.811c18a5fa6e80f03c8380da1785b09f.jpg',
 'xemay1267_jpg.rf.1913d7bc3f20a9ace9f5068c0feedcb7.jpg',
 'Cars414_png_jpg.rf.f4b022769aa023d8048d0c0499cfe302.jpg',
 'xemay1189_jpg.rf.5fea05d6667da5b28d2b4cf61efd534a.jpg',
 'xemayBigPlate62_jpg.rf.8906e34d787f2752331f3045ebc7cf85.jpg',
 'xemay2079_jpg.rf.6e934dd4aad90a746523eb7321d8e41e.jpg',
 'xemay820_jpg.rf.05ad669211b322014f8892326fbc94cb.jpg',
 'xemay2309_jpg.rf.c7ac1835b97bbee3315da8067ad0e33e.jpg',
 '01885b6c353314eb_jpg.rf.df33ff231773f744716546470980407a.jpg',
 'xemay343_jpg.rf.a7debc594b05ea089dd3268c9b54d33f.jpg',
 'xemay1435_jpg.rf.ddd444ac28a31e6bd379a6bd88307e1a.jpg',
 'xemay2253_jpg.rf.dcfd8b5d548dfc4e9e515a5461f0a76d.jpg',
 'xemay818_jpg.rf.616e02118cee7b7dd6852e15168d59ad.jpg',
 'CarL

In [8]:
img=cv2.imread(os.path.join('../dataset/test',img_names[0]))
results, rezied=char_model.detect(img.copy())

In [12]:
for img_name in img_names:
    img=cv2.imread(os.path.join('../dataset/test',img_name))
    if img_name.endswith('.jpg') is False:
        print(f'Skip file {img_name}')
        continue
    results, resized_img=char_model.detect(img.copy())
    for name,conf,box in results:
        resized_img=cv2.putText(resized_img, "{}".format(name), (int(box[0]), int(box[1])-3),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (255, 0, 255), 2)
        resized_img = cv2.rectangle(resized_img, (int(box[0]),int(box[1])), (int(box[2]),int(box[3])), (0,0,255), 1)
    if not os.path.exists(os.path.join('out')):
        os.makedirs(os.path.join('out'))
    cv2.imwrite(os.path.join('out',img_name),resized_img)
    print(f"Wrote to out/{os.path.join('out',img_name)}, {resized_img.shape}")

Wrote to out/out/Japanese-20License-20Plates-20-20Custom-20Japanese-20License-20P_jpg.rf.66abdc04fa057e3e07e95fab5caee67e.jpg, (640, 640, 3)
Wrote to out/out/xemay712_jpg.rf.c27e1277daa8f12d61c4d1e75377a3f0.jpg, (640, 640, 3)
Wrote to out/out/xemay2068_jpg.rf.d1cf718d5a7d6e7c335c386c89b362c2.jpg, (640, 640, 3)
Wrote to out/out/xemay2080_jpg.rf.811c18a5fa6e80f03c8380da1785b09f.jpg, (640, 640, 3)
Wrote to out/out/xemay1267_jpg.rf.1913d7bc3f20a9ace9f5068c0feedcb7.jpg, (640, 640, 3)
Wrote to out/out/Cars414_png_jpg.rf.f4b022769aa023d8048d0c0499cfe302.jpg, (640, 640, 3)
Wrote to out/out/xemay1189_jpg.rf.5fea05d6667da5b28d2b4cf61efd534a.jpg, (640, 640, 3)
Wrote to out/out/xemayBigPlate62_jpg.rf.8906e34d787f2752331f3045ebc7cf85.jpg, (640, 640, 3)
Wrote to out/out/xemay2079_jpg.rf.6e934dd4aad90a746523eb7321d8e41e.jpg, (640, 640, 3)
Wrote to out/out/xemay820_jpg.rf.05ad669211b322014f8892326fbc94cb.jpg, (640, 640, 3)
Wrote to out/out/Cars414_png_jpg.rf.f4b022769aa023d8048d0c0499cfe302.jpg, (640,

In [13]:
!pwd

/home/chisphung/ALPR_Fisheye/model
